In [28]:
import math
import gzip
import json
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import itertools
from functools import lru_cache
from itertools import chain, zip_longest
import pickle, random
from tqdm.notebook import tqdm
import copy
from glob import glob
from pprint import pprint
from copy import deepcopy
import pickle as pck
from functools import cache
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter

from utils import *

import matplotlib.style
import matplotlib as mpl
mpl.style.use('default')

title_font = 22
subplot_font = 14
plot_font = 18
legend_font = 16
col_font = subplot_font + 4
fig_dpi = 300

ylim_pad_factor = 0.05

# Same colours as mod-mini paper
scheme_to_color = {
    'DoubleDecyclingMinimizer': "black",
    'Miniception': "#00dd00",
    'Mod-minimizer': "blue",
    'Random minimizer': "red"
}

In [37]:
# Uniform plotting style
def style(ax, w=None):
    ax.grid(False)
    ax.grid(True, axis="x", color='#ccc',linewidth=0.5)
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(True)
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    if w:
        ax.spines['bottom'].set_visible(False)
        ax.set_ylim(ymin=1/w * (1-ylim_pad_factor), ymax=None)
        ax.axhline(1/w, label="$\\frac{1}{w}$", linestyle="dotted", linewidth=1, color='black')
    else:
        ax.spines['bottom'].set_visible(True)
        ax.set_ylim(ymin=0, ymax=0.8)

In [30]:

ax_width = 3
ax_height = 3
# ax_width, ax_height = ax_height, ax_width
fig, axs = plt.subplots(nrows=ax_height, ncols=ax_width)
fig.set_figheight(6)
fig.set_figwidth(12)
fig.subplots_adjust(wspace=.25)

w_range = list(range(2,5))
sigma = 2
for idx, w in enumerate(w_range):
    k_range = list(range(1, 15))
    my_densities = [aperiodic_bound(w, k, sigma) for k in k_range]
    suff_bounded = [aperiodic_bound_suff(w, k, sigma) for k in k_range]
    # ax = axs[int(idx / ax_width), idx % ax_width]
    ax = axs[idx, 0]
    style(ax, w)
    ax.plot(k_range, [ragnar_WABI_LB(w, k) for k in k_range], label="$\\frac{1.5}{w+k-0.5}$", color="red")
    ax.plot(k_range, [ragnar_ceil_LB(w, k) for k in k_range], label="$\\frac{\\lceil\\frac{w+k}{w}\\rceil}{w+k}$", color="blue")
    ax.plot(k_range, my_densities, label="$g_2$", color="green")
    ax.plot(k_range, suff_bounded, label="$g^\\prime_2$", color="green", linestyle="--", linewidth=1)
    opt_k = []
    opt_dens = []
    opt_color = []
    for k in k_range:
        if (w, k, sigma) in fwd_wksigma_to_dens:
            opt_k.append(k)
            opt_dens.append(fwd_wksigma_to_dens[(w, k, sigma)])
            opt_color.append("orange" if not aperiodic_bound_suff(w, k, sigma) == fwd_wksigma_to_dens[(w, k, sigma)] else "black")
    ax.scatter(opt_k, opt_dens, label="OPT", color=opt_color, s=25, zorder=100)
    
    prop={'size': subplot_font}
    ax.set_title(f"$w={w}$  ", loc="right", y=1.0, pad=-14, fontsize=subplot_font+2)
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    ax.set_ylim(ymax=(1+ylim_pad_factor)*my_densities[0])
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_font(prop)

    if w != w_range[-1]:
        ax.tick_params( axis='x',          # changes apply to the x-axis
                        which='both',      # both major and minor ticks are affected
                        bottom=False,      # ticks along the bottom edge are off
                        top=False,         # ticks along the top edge are off
                        labelbottom=False) # labels along the bottom edge are off
    else:
        ax.set_xlabel("$k$", color="black")

sigma_range = list(range(2, 5))
w = 2
for idx, sigma in enumerate(sigma_range):
    k_range = list(range(1, 15))
    my_densities = [aperiodic_bound(w, k, sigma) for k in k_range]
    suff_bounded = [aperiodic_bound_suff(w, k, sigma) for k in k_range]
    ax = axs[idx, 1]
    style(ax, w)
    # ax.plot(k_range, [1/w for k in k_range], label="$\\frac{1}{w}$", linestyle="dotted")
    ax.plot(k_range, [ragnar_WABI_LB(w, k) for k in k_range], label="$\\frac{1.5}{w+k-0.5}$", color="red")
    ax.plot(k_range, [ragnar_ceil_LB(w, k) for k in k_range], label="$\\frac{\\lceil\\frac{w+k}{w}\\rceil}{w+k}$", color="blue")
    ax.plot(k_range, my_densities, label="$g_\\sigma$", color="green")
    ax.plot(k_range, suff_bounded, label="$g^\\prime_\\sigma$", color="green", linestyle="--", linewidth=1)
    opt_k = []
    opt_dens = []
    opt_color = []
    for k in k_range:
        if (w, k, sigma) in fwd_wksigma_to_dens:
            opt_k.append(k)
            opt_dens.append(fwd_wksigma_to_dens[(w, k, sigma)])
            opt_color.append("orange" if not aperiodic_bound_suff(w, k, sigma) == fwd_wksigma_to_dens[(w, k, sigma)] else "black")
    ax.scatter(opt_k, opt_dens, label="OPT", color=opt_color, s=25, zorder=100)
    prop={'size': subplot_font}
    
    ax.set_title(f"$\\sigma={sigma}$  ", loc="right", y=1.0, pad=-14, fontsize=subplot_font+2)
    ax.set_ylim(ymax=(1+ylim_pad_factor)*my_densities[0])
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_font(prop)

    if sigma != sigma_range[-1]:
        ax.tick_params( axis='x',          # changes apply to the x-axis
                        which='both',      # both major and minor ticks are affected
                        bottom=False,      # ticks along the bottom edge are off
                        top=False,         # ticks along the top edge are off
                        labelbottom=False) # labels along the bottom edge are off
    else:
        ax.set_xlabel("$k$", color="black")

k=1
for idx, sigma in enumerate(sigma_range):
    w_range = list(range(2, 13))
    my_densities = [aperiodic_bound(w, k, sigma) for w in w_range]
    suff_bounded = [aperiodic_bound_suff(w, k, sigma) for w in w_range]
    
    ax = axs[idx, 2]
    style(ax)
    ax.plot(w_range, [1/w for w in w_range], label="$\\frac{1}{w}$", linestyle="dotted", linewidth=1,color='black')
    ax.plot(w_range, [ragnar_WABI_LB(w, k) for w in w_range], label="$\\frac{1.5}{w+k-0.5}$", color="red")
    ax.plot(w_range, [ragnar_ceil_LB(w, k) for w in w_range], label="$\\frac{\\lceil\\frac{w+k}{w}\\rceil}{w+k}$", color="blue")
    ax.plot(w_range, my_densities, label="$g_\\sigma$", color="green")
    ax.plot(w_range, suff_bounded, label="$g^\\prime_\\sigma$", color="green", linestyle="--", linewidth=1)
    opt_w = []
    opt_dens = []
    opt_color = []
    for w in w_range:
        if (w, k, sigma) in fwd_wksigma_to_dens:
            opt_w.append(w)
            opt_dens.append(fwd_wksigma_to_dens[(w, k, sigma)])
            opt_color.append("orange" if not aperiodic_bound_suff(w, k, sigma) == fwd_wksigma_to_dens[(w, k, sigma)] else "black")

    ax.scatter(opt_w, opt_dens, label="OPT", color=opt_color, s=25, zorder=100)
    prop={'size': subplot_font}
    # ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    ax.set_title(f"$\\sigma={sigma}$  ", loc="right", y=1.0, pad=-14, fontsize=subplot_font+1)
    # ax.set_ylim(((1-ylim_pad_factor)*(1/w), (1+ylim_pad_factor)*my_densities[0]))
    # ax.set_ylim(ymin=0)
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_font(prop)

    if sigma != sigma_range[-1]:
        ax.tick_params( axis='x',          # changes apply to the x-axis
                        which='both',      # both major and minor ticks are affected
                        bottom=False,      # ticks along the bottom edge are off
                        top=False,         # ticks along the top edge are off
                        labelbottom=False) # labels along the bottom edge are off
    else:
        ax.set_xlabel("$w$", color="black")

pad = 5
# for ax, sigma in zip(axs[:,0], sigma_range):
#     ax.annotate(f"$\\sigma={sigma}$", xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
#         xycoords=ax.yaxis.label, textcoords='offset points',
#         size='large', ha='right', va='center')

ax = axs[0][0]
ax.annotate(f"$\\sigma=2$", xy=(0.5, 1), xytext=(0, pad),
            xycoords='axes fraction', textcoords='offset points',
            size=col_font, ha='center', va='baseline')

ax = axs[0][1]
ax.annotate(f"$w=2$", xy=(0.5, 1), xytext=(0, pad),
            xycoords='axes fraction', textcoords='offset points',
            size=col_font, ha='center', va='baseline')

ax = axs[0][2]
ax.annotate(f"$k=1$", xy=(0.5, 1), xytext=(0, pad),
            xycoords='axes fraction', textcoords='offset points',
            size=col_font, ha='center', va='baseline')

handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, bbox_to_anchor=(.5, 0), loc='upper center', prop={'size': legend_font}, ncols=6)
# fig.supxlabel("$k$", y=.04, fontsize=plot_font)
fig.supylabel("Density", x=0.03, fontsize=plot_font)
fig.suptitle("Forward density lower bounds and ILP results", fontsize=title_font)
fig.savefig(f"media/bound-fwd-grid.svg", dpi=fig_dpi, bbox_inches='tight')
# fig.show()
plt.close(fig)


/home/philae/git/eth/git/minimizer-bound/scripts/utils.py:74: SymPyDeprecationWarning: 

The `sympy.ntheory.residue_ntheory.mobius` has been moved to `sympy.functions.combinatorial.numbers.mobius`.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-ntheory-symbolic-functions
for details.

This has been deprecated since SymPy version 1.13. It
will be removed in a future version of SymPy.

  return int((sum(mobius(d) * sigma ** (n // d) for d in divisors(n))) // n)


In [31]:
# Density table

fwd_wksigma_to_dens = {}
for pck_file in glob("fwd/dens/*.pck"):
    with open(pck_file, 'rb') as pck_in:
        fwd_wksigma_to_dens.update(pck.load(pck_in))
#print(len(fwd_wksigma_to_dens))
#pprint(sorted((fwd_wksigma_to_dens.keys())))

prev_sigma = 0
sigma_to_count = {sigma: sum(s == sigma for (w, k, s) in fwd_wksigma_to_dens) for sigma in range(5)}


for (w, k, sigma), dens in sorted(fwd_wksigma_to_dens.items(), key=lambda pair: (pair[0][2], pair[0][0], pair[0][1])):
    line = ""
    if sigma != prev_sigma:
        prev_w = 0
        # line += f"\multirow{{{sigma_to_count[sigma]}}}*{{{sigma}}}"
        print('---------')
    # else:
    #     line += "            "
    if w != prev_w:
        print("\\hline")
    prev_w = w
    line += f" {w:<2} & {k:<2} &  {float(dens):.4f} & {str(dens):<11} \\\\"
    prev_sigma = sigma
    print(line)

---------
\hline
 2  & 1  &  0.7500 & 3/4         \\
 2  & 2  &  0.6250 & 5/8         \\
 2  & 3  &  0.6250 & 5/8         \\
 2  & 4  &  0.5781 & 37/64       \\
 2  & 5  &  0.5781 & 37/64       \\
 2  & 6  &  0.5586 & 143/256     \\
 2  & 7  &  0.5586 & 143/256     \\
 2  & 8  &  0.5459 & 559/1024    \\
 2  & 9  &  0.5459 & 559/1024    \\
 2  & 10 &  0.5386 & 1103/2048   \\
 2  & 11 &  0.5386 & 1103/2048   \\
\hline
 3  & 1  &  0.5625 & 9/16        \\
 3  & 2  &  0.4688 & 15/32       \\
 3  & 3  &  0.4531 & 29/64       \\
 3  & 4  &  0.4375 & 7/16        \\
 3  & 5  &  0.4062 & 13/32       \\
 3  & 6  &  0.4023 & 103/256     \\
 3  & 7  &  0.4014 & 411/1024    \\
\hline
 4  & 1  &  0.4375 & 7/16        \\
 4  & 2  &  0.3750 & 3/8         \\
 4  & 3  &  0.3516 & 45/128      \\
 4  & 4  &  0.3398 & 87/256      \\
 4  & 5  &  0.3359 & 43/128      \\
\hline
 5  & 1  &  0.3594 & 23/64       \\
 5  & 2  &  0.3047 & 39/128      \\
 5  & 6  &  0.2734 & 35/128      \\
\hline
 6  & 1  &  0.2969 

In [32]:
def read(file):
    if file.endswith(".gz"):
        with gzip.open(file, 'rt', encoding='UTF-8') as f:
            data = json.load(f)
    else:
        with open(file, 'r') as f:
            data = json.load(f)
        
    df = pd.json_normalize(data)

    # Rename column 'tp.minimizer_type' to 'Minimizer type'
    df = df.rename(columns={'tp.minimizer_type': 'Minimizer type'})
    # In type column replace minizer value with random minimizer.
    df['Minimizer type'] = df['Minimizer type'].replace('Minimizer', 'Random minimizer')
    df['Minimizer type'] = df['Minimizer type'].replace('LrMinimizer', 'LR-minimizer')
    df['Minimizer type'] = df['Minimizer type'].replace('ModMinimizer', 'Mod-minimizer')
    df['Minimizer type'] = df['Minimizer type'].replace('MiniceptionNew', 'Modified miniception')

    df['param'] = df['tp.k0'].fillna(0) #+ df['tp.r'].fillna(0)

    return df

In [33]:
all_sigmas = [2, 4, 256]
df_by_sigma = []
for sigma in all_sigmas:
    datafile = f"data/practical_{sigma}.json.gz"
    df_by_sigma.append(read(datafile))

df_random = pd.concat(df_by_sigma).sort_values("k")
df_random["text"] = "Random"

datafile = f"data/practical_chm13Y.json.gz"
df_chm13 = read(datafile)
df_chm13["text"] = "ChrY"

df = pd.concat([df_random, df_chm13]).sort_values("k")
all_window_sizes = sorted(df["w"].unique())
all_scheme_names = sorted(df["Minimizer type"].unique())
pprint(sorted(all_scheme_names))

scheme_names = [
    # 'AltRotMinimizer',
    # 'BdAnchor',
    # 'DecyclingMinimizer',
    'DoubleDecyclingMinimizer',
    # 'LR-minimizer',
    'Miniception',
    'Mod-minimizer',
    # 'ModSampling',
    # 'Modified miniception',
    # 'OpenClosedSyncmerMinimizer',
    # 'OpenSyncmerMinimizer',
    'Random minimizer',
    # 'RotMinimizer'
]
# scheme_names = all_scheme_names
df_schemes = df[df["Minimizer type"].isin(scheme_names)].sort_values("k")

['DoubleDecyclingMinimizer',
 'Miniception',
 'Mod-minimizer',
 'Modified miniception',
 'OpenClosedSyncmerMinimizer',
 'Random minimizer']


In [34]:
sigma = 4
w = 5
k = 31
text = "Random"
for w, k in [(5, 31), (10, 15), (19, 19), (12,20)]:
    df_comp = df_schemes[
    (df_schemes["w"] == w) 
    & (df_schemes["sigma"] == sigma) 
    & (df_schemes["k"] == k)
    & (df_schemes["text"] == text)]
    best = df_comp.loc[df_comp["density"].idxmin()]
    best_density = best["density"]
    best_scheme = best["Minimizer type"]
    lower_bound = aperiodic_bound_suff(w, k, sigma)
    
    print(f"For w={w}, k={k}, {best_scheme} performed best w/ a density of {best_density:.4f}.")
    print(f"Gap decreased from {100*(best_density-(1/w))/(1/w):.2f}% --> {100*(best_density-lower_bound)/lower_bound:.2f}%")
    print()

For w=5, k=31, Mod-minimizer performed best w/ a density of 0.2259.
Gap decreased from 12.94% --> 1.65%

For w=10, k=15, Mod-minimizer performed best w/ a density of 0.1434.
Gap decreased from 43.39% --> 11.12%

For w=19, k=19, DoubleDecyclingMinimizer performed best w/ a density of 0.0790.
Gap decreased from 50.05% --> 2.67%



ValueError: attempt to get argmin of an empty sequence

In [35]:
w_range = [5, 10, 19]
k_min, k_max = 6, 33
plot_ratio = False
sigma = 4

title_font = 18
subplot_font = 14
plot_font = 16
legend_font = 14
fig_dpi = 300

fig, axs = plt.subplots(nrows=len(w_range), ncols=1)
fig.set_figheight(6)
fig.set_figwidth(6)

for idx, w in enumerate(w_range):
    df_sub = df_schemes[(df_schemes["w"] == w) & (df_schemes["sigma"] == sigma)]
    ax = axs[idx]
    style(ax, w)
    
    for scheme in scheme_names:
        for text in ["Random", "ChrY"]:
            df_this = df_sub[(df_sub["Minimizer type"] == scheme) & (df_sub["text"] == text)]
            if plot_ratio:
                ratio = df_this.apply(lambda row: 100*((row["density"] - (1/w)) / (1/w)), axis=1)
                y_vals = ratio
            else:
                y_vals = df_this["density"]
                
            if text == "Random":
                ax.plot(df_this["k"], y_vals, label=scheme, color=scheme_to_color[scheme] if scheme in scheme_to_color else None)
            else:
                # ax.plot(df_this["k"], y_vals, color=scheme_to_color[scheme], linewidth=4, alpha=.2)
                ax.plot(df_this["k"], y_vals, color=scheme_to_color[scheme], alpha=0.3)
                # ax.scatter(df_this["k"], y_vals, color=scheme_to_color[scheme], s=10, alpha=0.4)

    if not plot_ratio:
        # Plot lower bounds
        # ax.axhline(1/w, color="black", linestyle="--", linewidth=1, label="$\\frac{1}{w}$")
        k_range = list(range(k_min, k_max))
        g_prime = [aperiodic_bound_suff(w, k, sigma) for k in k_range] 
        ax.plot(k_range, [ragnar_WABI_LB(w, k) for k in k_range], label="$\\frac{1.5}{w+k-0.5}$", color="red", linestyle="--", linewidth=1.5)
        ax.plot(k_range, g_prime, label=f"$g^\\prime_\\sigma$", color="green", linestyle="--", linewidth=1.5)

        # Pad upper and lower limits
        ax.set_ylim((.9/w, 2.3 / (w+1)))
        ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    else:
        ax.set_yscale("log")
    ax.set_xlim((k_min, k_max))
    ax.xaxis.set_major_locator(MaxNLocator(integer=True,nbins=8))

    # Add label slightly below top
    ax.set_title(f"$w={w}$", y=1.0, pad=-14)
    
    prop={'size': subplot_font}    
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_font(prop)  
   

    # Turn off x-axis items for all axes except last
    if w != w_range[-1]:
        ax.tick_params( axis='x',          # changes apply to the x-axis
                        which='both',      # both major and minor ticks are affected
                        bottom=False,      # ticks along the bottom edge are off
                        top=False,         # ticks along the top edge are off
                        labelbottom=False) # labels along the bottom edge are off

handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='center right', bbox_to_anchor=(1.55, .5), prop={'size': legend_font}, ncols=1)
fig.supxlabel("$k$", y=.03, fontsize=plot_font)
if plot_ratio:
    fig.suptitle(f"Gap between $g_{{{sigma}}}$ and existing schemes", fontsize=title_font)
    fig.supylabel(f"Gap ratio (%)", x=0, fontsize=plot_font)
else:
    fig.suptitle(f"Existing schemes vs. lower bounds ($\\sigma={{{sigma}}}$)", fontsize=title_font)
    fig.supylabel(f"$d(f)$", x=0, fontsize=plot_font)

fig.savefig(f"media/benchmark-sigma{sigma}-{'ratio' if plot_ratio else 'density'}.svg", dpi=fig_dpi, bbox_inches='tight')
# fig.show()
plt.close(fig)


/home/philae/git/eth/git/minimizer-bound/scripts/utils.py:74: SymPyDeprecationWarning: 

The `sympy.ntheory.residue_ntheory.mobius` has been moved to `sympy.functions.combinatorial.numbers.mobius`.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-ntheory-symbolic-functions
for details.

This has been deprecated since SymPy version 1.13. It
will be removed in a future version of SymPy.

  return int((sum(mobius(d) * sigma ** (n // d) for d in divisors(n))) // n)


In [36]:
w_range = all_window_sizes
sigma_range = all_sigmas
k_min, k_max = 6, 64
plot_ratio = False

title_font = 23
subplot_font = 14
plot_font = 18
legend_font = 14
fig_dpi = 300

fig, axs = plt.subplots(nrows=len(w_range), ncols=len(all_sigmas))
fig.set_figheight(10)
fig.set_figwidth(12)
text = "Random"

for idx_sigma, sigma in enumerate(sigma_range):
    for idx_w, w in enumerate(w_range):
        df_sub = df_schemes[
            (df_schemes["w"] == w) 
            & (df_schemes["sigma"] == sigma)
            & (df_schemes["text"] == text)
        ]
        ax = axs[idx_w, idx_sigma]
        style(ax, w)
        
        for scheme in scheme_names:
            df_this = df_sub[(df_sub["Minimizer type"] == scheme) & (df_sub["text"] == text)]
            if plot_ratio:
                ratio = df_this.apply(lambda row: 100*((row["density"] - (1/w)) / (1/w)), axis=1)
                y_vals = ratio
            else:
                y_vals = df_this["density"]
                
            if text == "Random":
                ax.plot(df_this["k"], y_vals, label=scheme, color=scheme_to_color[scheme] if scheme in scheme_to_color else None)
            else:
                ax.plot(df_this["k"], y_vals, color=scheme_to_color[scheme], linewidth=4, alpha=.2)
    
        if not plot_ratio:
            # Plot lower bounds
            # ax.axhline(1/w, color="black", linestyle="--", linewidth=1, label="$\\frac{1}{w}$")
            k_range = list(range(k_min, k_max))
            g_prime = [aperiodic_bound_suff(w, k, sigma) for k in k_range] 
            ax.plot(k_range, [ragnar_WABI_LB(w, k) for k in k_range], label="$\\frac{1.5}{w+k-0.5}$", color="red", linestyle="dashed", linewidth=1.5)
            ax.plot(k_range, g_prime, label=f"$g^\\prime_\\sigma$", color="green", linestyle="--", linewidth=1.5)
    
            # Pad upper and lower limits
            ax.set_ylim((.9/w, 2.3 / (w+1)))
            ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        else:
            ax.set_yscale("log")
        ax.set_xlim((k_min, k_max))
        # ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        
        prop={'size': subplot_font}    
        for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
            item.set_font(prop)  
       
        # Turn off x-axis items for all axes except last
        if w != w_range[-1]:
            ax.tick_params( axis='x',          
                            which='both',      
                            bottom=False,      
                            top=False,         
                            labelbottom=False) 

        # Turn off y-axis items for all axes except left
        if sigma != sigma_range[0] and not plot_ratio:
            ax.tick_params( axis='y',          
                            which='both',      
                            bottom=False,     
                            left=False,        
                            labelleft=False) 

# Add labels to rows and columns
axis_pad = 5
for ax, w in zip(axs[:,0], w_range):
    ax.annotate(f"$w={w}$", xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - axis_pad, 0),
        xycoords=ax.yaxis.label, textcoords='offset points',
        size='large', ha='right', va='center')
    
for ax, sigma in zip(axs[0], all_sigmas):
    ax.annotate(f"$\\sigma={sigma}$", xy=(0.5, 1), xytext=(0, axis_pad),
                xycoords='axes fraction', textcoords='offset points',
                size='large', ha='center', va='baseline')


handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, bbox_to_anchor=(0.5, 0), loc='center', prop={'size': legend_font}, ncols=4)
fig.supxlabel("$k$", y=.04, fontsize=plot_font)
if plot_ratio:
    fig.suptitle(f"Gap between $g_{{{sigma}}}$ and existing schemes", fontsize=title_font)
    fig.supylabel(f"gap ratio", x=0, fontsize=plot_font)
else:
    fig.suptitle(f"Existing schemes vs. lower bounds", fontsize=title_font)
    fig.supylabel(f"$d(f)$", x=-0.02, fontsize=plot_font)

fig.savefig(f"media/benchmark-grid-{'ratio' if plot_ratio else 'density'}.svg", dpi=fig_dpi, bbox_inches='tight')
# fig.show()
plt.close(fig)